# Machine Learning with `scikit-learn`

Modern causal inference is a blend of causal theory, graphical models, mathematical statistics and machine learning. To use causal inference techniques, one needs to have a solid understanding of these fields to ensure accurate estimation of causal effects.

The most popular causal ML packages in Python, including `econml`, `causallib`, `doubleml`, and `causalml`, all utilize `scikit-learn` as their backend ML library. 

This section is a quick introduction to fitting models with `scitkit-learn` so that we can understand how `doubleml` is used in the next section.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid") 
sns.set_palette('viridis')
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['font.family'] = 'monospace'

### Machine Learning
## Cross validation and data resampling
from sklearn.model_selection import train_test_split

## Modeling
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

## Model Evaluation
from sklearn.metrics import (
    confusion_matrix,
    precision_score, 
    recall_score,
    f1_score,
    classification_report)

In [ ]:
# Load observational dataset
observational_df = pd.read_pickle('../data/observational_df.pkl').drop(columns=['upsell_marketing'])

# Identify columns
customer_features = observational_df.drop(columns=['converted']).columns.to_list()
target_outcome = 'converted'

print('Customer Features: ', customer_features)

observational_df.head(5)

## Basic Machine Learning Workflow
Below is a commonly used workflow for training machine learning algorithms. The key is to randomly split our dataset into a training and test set to ensure optimal performance on new datasets

<br>
<center>
<img 
  src="../assets/model_process.png" 
  alt="Modeling Process" 
  style="width:auto;height:300px;"
> 
<br>
<br>

## Data Resampling
Splitting our data into a training and test set is important for evaluating model performance and estimating generalization to new data

- **Under-fitting**
    - Model can't capture complex trends in the data
    - Give away - poor performance on both training and test datasets

<br>
<br>
<center>
<img 
  src="../assets/under_fitting.png" 
  alt="Underfitting" 
  style="width:auto;height:375px;"
> 
<br>
<br>

<br>

- **Over-fitting**
    - Model finds trends that don't exist
    - Give away - great performance training data and *poor performance test dataset*

<br>
<br>

<center>
<img 
  src="../assets/over_fitting.png" 
  alt="Underfitting" 
  style="width:auto;height:375px;"
> 
<br>
<br>

### Optimal Complexity

Generally, as we go from simple models to more complex:
- Training error continues to decrease (potentially reaching zero!)
- Test error decreases initially, but increases when we are over-fitting
- Goal is to find the optimal model complexity to ensure good performance on new data

<br>
<br>

<center>
<img 
  src="../assets/optimal_complexity.png" 
  alt="Underfitting" 
  style="width:auto;height:375px;"
> 
<br>
<br>

### Creating Training and Test Datasets with `scikit-learn`
The `train_test_split` function can randomly divide our original dataset into train and test sets

In [ ]:
train_df, test_df = train_test_split(
    observational_df, 
    train_size = 0.7, 
    stratify=observational_df[target_outcome])

# Check dataset properties
print(
    f'Training Rows: {train_df.shape[0]:,}',
    f'Training Signup Rate: {train_df[target_outcome].mean():.1%}',
    f'Test Rows: {test_df.shape[0]:,}',
    f'Test Signup Rate: {test_df[target_outcome].mean():.1%}',
    sep='\n')

## Modeling with `scikit-learn`

### Introduction to Classification

In classification, we are predicting a target outcome with categorical values. Typically the category of interest which we are predicting is labeled as the **positive class**. 

The positive class for our data would be 1. indicating a signup. The remaining outcome category is the negative class (0 in our data).

<br>
<center>
<img 
  src="../assets/classification.png" 
  alt="Classification" 
  style="width:auto;height:500px;"
> 
<br>

#### Finding the Optimal Decision Boundary

The goal of classification is to find an optimal decision boundary which splits the feature space into two distinct regions, one where we predict the positive class and the other the negative class. 

The decision boundary is a function where the estimated probability of the positive outcome is equal to 0.5. 

Feature combinations above this line have $Prob(Outcome = Positive \hspace{0.5em} class) > 0.5$ and are classified as a positive outcome.

The decision boundary below is a linear boundary. Models such as **Logistic Regression** form linear decision boundaries.

<br>
<center>
<img 
  src="../assets/classification_boundary.png" 
  alt="Classification Boundary" 
  style="width:auto;height:500px;"
> 
<br>

The great about `scikit-learn` is that each model is defined as its own object with `fit()` and `predict()` methods. Once a model has been specified, then the syntax for training and performance evaluation remains the same.

Another benefit of using `scikit-learn` for machine learning is its amazing [documentation](https://scikit-learn.org/stable/index.html)

### Model Training

In [ ]:
# Define the model object
logistic_model = LogisticRegression()

# Train with the fit() method
logistic_model.fit(
    X=train_df[customer_features],
    y=train_df[target_outcome]
);

<br>

We can now use our trained logistic regression model to generate predictions. In classification, we are interested in two types of output:
- The estimated probability score for each class of the target variable
    - This would be the estimated probability of a 1 and 0 outcome in our data (`amu_signup`)
    - This is done with the `predict_proba()` method
- Predicted target outcomes
    - This would be an array of 1s and 0s based on a probability threshold or cut-off value (default is 0.5)
    - This done with the `predict()` method

In [ ]:
# Predicted outcome values
logistic_model.predict(
    X=train_df[customer_features])[0:10]

In [ ]:
# Estimated probabilities for each possible outcome [0, 1] order
logistic_model.predict_proba(
    X=train_df[customer_features])[0:10]

<br>

How do we know which target outcome category each column corresponds to in the `predict_proba()` output?
We have to use the `.class_` attritube of our trained logistic regression model.

In [ ]:
logistic_model.classes_

### Model Evaluation

Let's combine our original test dataset and the model predictions into a single data frame. This would be usefully for detailed analysis of our model performance

In [ ]:
test_df_results = (
    test_df
    .assign(
        converted_pred=logistic_model.predict(X=test_df[customer_features]),
        prob_0=logistic_model.predict_proba(X=test_df[customer_features])[:, 0],
        prob_1=logistic_model.predict_proba(X=test_df[customer_features])[:, 1],
    )
)

test_df_results

### Confusion Matrix

<br>
<center>
<img 
  src="../assets/confusion_matrix_1.png" 
  style="width:auto;height:350px;"
> 

<br>
<br>

<center>
<img 
  src="../assets/confusion_matrix_2.png"  
  style="width:auto;height:220px;"
> 
<br>

In [ ]:
# Confusion matrix 
confusion_matrix(
    y_true=test_df_results['converted'],
    y_pred=test_df_results['converted_pred'],
    labels=[1, 0] # Order the rows and columns
)

#### Recall

To remember the meaning of this metric:
- Recall starts with an "R"
- Model accuracy for the "real" positive outcomes
    - For us this is the model accuracy among the customers who **actually signed up (converted)**
    - Equal to 17/(17 + 2641) = 0.64% (from our confusion matrix above)

<br>
<center>
<img 
  src="../assets/recall.png" 
  style="width:auto;height:350px;"
> 

In [ ]:
# With scikit-learn
recall_est = recall_score(
    y_true=test_df_results['converted'],
    y_pred=test_df_results['converted_pred'],
    pos_label=1
)

print(f'{recall_est:.2%}')

#### Precision

To remember the meaning of this metric:
- Precision starts with an "P"
- Model accuracy for the "predicted" positive outcomes
    - For us this is the model accuracy among the customers who were **predicted to convert**
    - Equal to 17/(17 + 22) = 43.59% (from our confusion matrix above)

<br>
<center>
<img 
  src="../assets/precision.png" 
  style="width:auto;height:350px;"
> 

In [ ]:
precision_est = precision_score(
    y_true=test_df_results['converted'],
    y_pred=test_df_results['converted_pred'],
    pos_label=1
)

print(f'{precision_est:.2%}')

#### F-1 Score

<br>
<center>
<img 
  src="../assets/f1_score.png" 
  style="width:auto;height:330px;"
> 

In [ ]:
f1_est = f1_score(
    y_true=test_df_results['converted'],
    y_pred=test_df_results['converted_pred'],
    pos_label=1
)

print(f'{f1_est:.2%}')

### Model Performance Within Each Outcome Class

It is also important to look at precision, recall, and f-1 **within** each outcome category. This is done with the `classification_report()` function.

In [ ]:
print(
    classification_report(
        y_true=test_df_results['converted'],
        y_pred=test_df_results['converted_pred'],
        labels=[0, 1] # Row order
        )
)

### Random Forest Classifier
Could we get better performance with a more complex ML algorithm? Let's try a random forest

In [ ]:
# Define the model object
rf_model = RandomForestClassifier(
    class_weight='balanced_subsample',
    min_samples_split=10,
    n_estimators=500,
    random_state=42
)

# Train with the fit() method
rf_model.fit(
    X=train_df[customer_features],
    y=train_df[target_outcome]
);

Looks like we are moving in the right direction! Our recall score among the positive samples increased from < 1% to 42%. Our estimated F1 score increased from 1.26% to 29%

In [ ]:
# Performance on the test data
print(
    classification_report(
        y_true=test_df[target_outcome],
        y_pred=rf_model.predict(test_df[customer_features]),
        labels=[0, 1] # Row order
        )
)